In [5]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [6]:
import pandas
import pandas as pd
import numpy as np
import gensim
import csv
import ProbData
from ProbModels import *
import pickle
from utilities import *
from sentences import *
from DCS import *
import os
from sklearn.linear_model import LogisticRegression
from sklearn.cross_validation import KFold

In [10]:
df = pd.read_csv('pcrw_24.csv', usecols=['f', 'ln_lemma', 'rn_lemma', 'ln_cng', 'rn_cng', '111',
       '112', '113', '121', '122', '123', '131', '132', '133', '211', '212',
       '213', '221', '222', '223', '231', '232', '233', '311', '312', '313',
       '321', '322', '323', '331', '332', '333', '400', '500', '600', '700',
       '800', '900', 'flag', '011', '022', '033'])

df = df.drop(df[np.isnan(df.ln_cng) | np.isnan(df.rn_cng)].index) # Add

df.ln_cng = df.ln_cng.astype(int) # Add
df.rn_cng = df.rn_cng.astype(int) # Add

In [12]:
df.columns

Index(['f', 'ln_lemma', 'rn_lemma', 'ln_cng', 'rn_cng', '111', '112', '113',
       '121', '122', '123', '131', '132', '133', '211', '212', '213', '221',
       '222', '223', '231', '232', '233', '311', '312', '313', '321', '322',
       '323', '331', '332', '333', '400', '500', '600', '700', '800', '900',
       'flag', '011', '022', '033'],
      dtype='object')

In [13]:
df.head(6)

,f,ln_lemma,rn_lemma,ln_cng,rn_cng,111,112,113,121,122,...,400,500,600,700,800,900,flag,011,022,033
0,123274.p2,arjuna,vaDa,149,35,0.000223,0.000053,0.0,0,0.000047,...,0.0,0.0,0.0,0.0,0.0,0,1,0.00647,0,0.0
1,123274.p2,arjuna,vaDa,151,35,0.000223,0.000053,0.0,0,0.000047,...,0.0,0.0,0.0,0.0,0.0,0,1,0.00647,0,0.0
2,123274.p2,arjuna,vaDa,149,36,0.000223,0.000055,0.0,0,0.000020,...,0.0,0.0,0.0,0.0,0.0,0,1,0.00647,0,0.0
3,123274.p2,arjuna,vaDa,151,36,0.000223,0.000055,0.0,0,0.000020,...,0.0,0.0,0.0,0.0,0.0,0,1,0.00647,0,0.0
4,123274.p2,arjuna,vaDa,149,75,0.000223,0.000034,0.0,0,0.000128,...,0.0,0.0,0.0,0.0,0.0,0,1,0.00647,0,0.0
5,123274.p2,arjuna,vaDa,151,75,0.000223,0.000034,0.0,0,0.000128,...,0.0,0.0,0.0,0.0,0.0,0,1,0.00647,0,0.0


In [14]:
pb = ProbModels(fullCo_oc_mat = ProbData.fullCo_oc_mat, unigram_counts = ProbData.unigram_counts,
               cng2cngFullMat = ProbData.cng2cngFullMat, cng2index_dict = ProbData.cng2index_dict,
               w2w_samecng_fullmat=ProbData.w2w_samecng_fullmat, samecng_unigram_counts=ProbData.samecng_unigram_counts,
               v2c_fullMat = ProbData.v2c_fullMat)

In [21]:
df['011'] = row_11
df['022'] = row_22
df['033'] = row_33

In [16]:
# hList = ['f', 'ln_lemma', 'rn_lemma', 'ln_cng', 'rn_cng']
hList = []
for u in range(1,10):
    if u < 4:
        for v in [1,2,3]:
            for w in [1,2,3]:
                hList.append(u*100 + 10*v + w)
    else:
        hList.append(u*100)
hList.extend(['011','022','033'])

# hList.append('flag')

goodCols = []
for h in hList:
    if np.min(df[str(h)]) == 0 and np.max(df[str(h)]) == 0:
#         print(h)
        pass
    else:
        goodCols.append(str(h))
        
print(goodCols)

['111', '112', '113', '122', '123', '131', '132', '211', '212', '213', '222', '223', '231', '232', '311', '312', '322', '400', '500', '600', '700', '800', '011', '033']


In [17]:
df.flag.value_counts()

0    4283777
1     801548
Name: flag, dtype: int64

In [8]:
n_indices = list(range(4300000))
n_indices = n_indices[0:4004008:5]
print(len(n_indices), max(n_indices))

800802 4004005


In [18]:
df_n = df[df['flag'] == 0]
df_p = df[df['flag'] == 1]

df_n = df_n.sample(df_p.shape[0])
print(df_n.shape, df_p.shape)

(801548, 42) (801548, 42)


In [19]:
frames = [df_n, df_p]
df_2 = pd.concat(frames)
df_2 = df_2.sample(df_2.shape[0])

In [21]:
print(df_2[df_2['flag'] == 0].shape)
print(df_2[df_2['flag'] == 1].shape)

(801548, 42)
(801548, 42)


In [22]:
df_2 = df_2.reset_index(drop=True)

In [23]:
df_2.head()

,f,ln_lemma,rn_lemma,ln_cng,rn_cng,111,112,113,121,122,...,400,500,600,700,800,900,flag,011,022,033
0,143026.p2,SAlva,nagara,29,69,0.000265,0.014413,0.000000,0,0.116123,...,0.25,0.0,0.0,0.0,0.0,0,1,0.027211,0,0.0
1,14307.p2,kzatra,taqAga,56,3,0.000013,0.041058,0.000013,0,0.236454,...,0.00,0.0,0.0,0.0,0.0,0,0,0.000000,0,0.0
2,2846.p2,eva,prARa,2,29,0.000653,0.050239,0.000069,0,0.511285,...,0.00,0.0,0.0,0.0,0.0,0,1,0.005674,0,0.0
3,53544.p2,tad,tad,71,36,0.017795,0.000091,0.000008,0,0.000831,...,0.00,0.0,0.0,0.0,0.0,0,1,0.000000,0,0.0
4,136470.p2,ala,DAraRa,56,36,0.000000,0.000000,0.000000,0,0.000000,...,0.00,0.0,0.0,0.0,0.0,0,0,0.000000,0,0.0


In [24]:
lr = LogisticRegression(solver='lbfgs', fit_intercept=False) # lbfgs, No Bias
# lr.fit()
kf = KFold(df_2.shape[0],4)

In [ ]:
coeffs = []
for train, test in kf:
    lr.fit(df_2.loc[train, goodCols], df_2.loc[train, 'flag'])
    print(np.sum(df_2.loc[test, 'flag'] == lr.predict(df_2.loc[test, goodCols]))/len(test))
    coeffs.append(lr.coef_)

In [15]:
df_results = pd.DataFrame(columns=['Path', 'CV_1', 'CV_2' ,'CV_3', 'CV_4'])
for i in range(len(goodCols)):
    df_results.loc[i] = [goodCols[i], coeffs[0][0][i], coeffs[1][0][i], coeffs[2][0][i], coeffs[3][0][i]]

In [16]:
df_results['avg'] = np.sum(df_results[['CV_1', 'CV_2', 'CV_3', 'CV_4']], axis = 1)/4
df_results['min'] = np.min(df_results[['CV_1', 'CV_2', 'CV_3', 'CV_4']], axis = 1)

In [17]:
np.asarray(df_results.loc[df_results['min'] > 0, 'avg'])

array([  1.80602478,   0.25526765,  11.02924932,   2.14424648,
         5.49015577,  36.3877483 ,   9.94486031,   8.52550359,
         4.27411239,  13.04355219,   9.85612646,   0.25621379,
         1.26833802,   3.23112315,   0.31753472,   0.36231652,
         1.80477853,   0.39601337])

In [19]:
df_results

,Path,CV_1,CV_2,CV_3,CV_4,avg,min
0,111,1.732029,1.710357,1.793793,1.987920,1.806025,1.710357
1,112,0.254697,0.137528,0.339358,0.289486,0.255268,0.137528
2,113,10.730331,11.317838,10.519727,11.549102,11.029249,10.519727
3,122,2.117218,2.160538,2.155403,2.143828,2.144246,2.117218
4,123,5.350218,5.256214,5.826620,5.527571,5.490156,5.256214
5,131,36.446636,37.445279,36.089858,35.569219,36.387748,35.569219
6,132,-1.500415,-1.569429,-1.679975,-1.094609,-1.461107,-1.679975
7,211,-10.699556,-10.872726,-10.035713,-11.220252,-10.707062,-11.220252
8,212,-4.423943,-4.416033,-4.591167,-4.324654,-4.438949,-4.591167
9,213,9.585790,10.347819,10.148123,9.697709,9.944860,9.585790


In [18]:
def getname(c):
    if c == '1':
        return 'simple lemma'
    if c == '2':
        return 'simple cng'
    if c == '3':
        return 'simple word'
    if c == '4':
        return 'noun-noun'
    if c == '5':
        return 'verb-gerund'
    if c == '6':
        return 'verb-absol'
    if c == '7':
        return 'verb-ppp'
    if c == '8':
        return 'verb-inf'
    if c == '9':
        return 'verb-ppa'
df_path = pd.DataFrame(columns=['Path Code', 'Element 1', 'Element 2', 'Element 3'])
for i in range(len(goodCols)):
    path = goodCols[i]
    df_path.loc[i] = [path, getname(path[0]), getname(path[1]), getname(path[2])]

In [19]:
df_path

,Path Code,Element 1,Element 2,Element 3
0,111,simple lemma,simple lemma,simple lemma
1,112,simple lemma,simple lemma,simple cng
2,113,simple lemma,simple lemma,simple word
3,122,simple lemma,simple cng,simple cng
4,123,simple lemma,simple cng,simple word
5,131,simple lemma,simple word,simple lemma
6,132,simple lemma,simple word,simple cng
7,211,simple cng,simple lemma,simple lemma
8,212,simple cng,simple lemma,simple cng
9,213,simple cng,simple lemma,simple word


In [35]:
goodWeights = np.array([0.30758324,  12.0556656 ,   2.17109444,   5.78959942, 33.49313423,  10.32264846,   7.94864612,   4.31576204, 13.58834529,   9.17713191,   0.24156488,   1.22543121, 3.23680681,   0.33608101,   0.39159008,   1.72731087,   0.36444531])


In [36]:
goodWeights = goodWeights/np.sum(goodWeights)

In [37]:
goodWeights

array([ 0.00288289,  0.11299414,  0.02034902,  0.05426418,  0.3139211 ,
        0.09675109,  0.07450028,  0.04045034,  0.12735949,  0.08601451,
        0.00226412,  0.0114856 ,  0.03033762,  0.00314999,  0.00367026,
        0.01618957,  0.00341584])

In [3]:
w3 = np.array([0.3, 0.1, 0.55])
w3 *= 0.4

In [4]:
w3

array([ 0.12,  0.04,  0.22])

In [40]:
np.sum(goodWeights*0.6) + np.sum(w3)

0.97999999999999998

In [41]:
np.sum(w3)

0.38

In [3]:
goodCols = ['111', '112', '113', '122', '123', '131', '132', '211', '212', '213', '222', '223', '231', '232', '311', '312', '322', '400', '500', '600', '700', '800']

In [23]:
x = np.asarray(df_results['avg'])

In [29]:
z = x/(np.max(x) - np.min(x))

In [30]:
z

array([ 0.0383487 ,  0.00542029,  0.23419246,  0.04553042,  0.11657666,
        0.7726488 , -0.0310248 , -0.2273512 , -0.0942556 ,  0.2111668 ,
       -0.07864587,  0.18102852,  0.09075549,  0.27696369,  0.20928265,
        0.00544038,  0.02693159,  0.0686089 ,  0.00674246,  0.00769334,
        0.03832224,  0.00840885])

In [31]:
np.sum(z)

1.9127847672015557